In [2]:
import duckdb
import pandas as pd 
from pathlib import Path

# handling the path, i show where the database and sql file is 
duckdb_path = "../data/sakila.duckdb"
sql_load_path = "../sql/load_sakila.sql"

# Cleans old datafile if it exists, so thar we start a new every time
Path(duckdb_path).unlink(missing_ok = True)

# here i use the with statement to make sure the connection is closed after use
# i also  do the connection and the readiing of the sql file in the same line as Debbie did
# to ensure both are properly closed after use
with duckdb.connect(duckdb_path) as conn, open(sql_load_path, 'r') as ingest_script:
    # this runs the sql script that connects to the file 
    conn.execute(ingest_script.read())

    # This will import tables to pandas
    # also when the connection is open we can run sql queries, and also convert them to Pandas Data frames
    print("load the tables into memory")
    actor = conn.sql("SELECT * FROM actor").df()
    category = conn.sql("SELECT * FROM category").df()
    film = conn.sql("SELECT * FROM film").df()
    film_actor = conn.sql("SELECT * FROM film_actor").df()
    film_category = conn.sql("SELECT * FROM film_category").df()

#this verifies that the data has been loaded correctly by printing the first 5 rows of each table
# so that i cn ensure myself that the data has been loaded correctly
print(" data has been loaded correctly,")

print(f"testing it out. First movie: {film['title'][0]}")







    



























load the tables into memory
 data has been loaded correctly,
testing it out. First movie: ACADEMY DINOSAUR
